In [37]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


In [38]:
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [5]:
# 最普通常规写法
batch_size=10
x=tf.placeholder(tf.float32,shape=[None,784])
w=tf.Variable(tf.zeros([784,10]))
bias=tf.Variable(tf.zeros([10]))
y_=tf.placeholder(tf.float32,shape=[None,10])
y=tf.matmul(x,w)+bias
y=tf.nn.softmax(y)

cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

for i in range(1000):
    batch_xs,batch_ys=mnist.train.next_batch(batch_size)
    train_step.run({x:batch_xs,y_:batch_ys})

correction_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correction_prediction,tf.float32))

print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels}))

0.8762


In [21]:
#神经网络版
x = tf.placeholder(tf.float32,shape=[None,784])
y_ =tf.placeholder(tf.float32,shape=[None,10])
x_image=tf.reshape(x,[-1,28,28,1])



def weight_Variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_Variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_conv1=weight_Variable([5,5,1,32])
bias_conv1=bias_Variable([32])
# 第一层卷积
conv_1=tf.nn.relu(conv2d(x_image,W_conv1)+bias_conv1)
pool_1=max_pool_2x2(conv_1)

# 第二层
W_conv2=weight_Variable([3,3,32,64])
bias_conv2=bias_Variable([64])
conv_2=tf.nn.relu(conv2d(pool_1,W_conv2)+bias_conv2)
pool_conv2=max_pool_2x2(conv_2)

# 平铺1
W_fc1=weight_Variable([7*7*64,1024])
bias_fc1=bias_Variable([1024])
pool_fc1=tf.reshape(pool_conv2,[-1,7*7*64])
fc1=tf.nn.relu(tf.matmul(pool_fc1,W_fc1) + bias_fc1)
# 防止过拟合丢弃一部分隐藏层
fc1_droput=tf.nn.dropout(fc1,0.5)

# 平铺2
W_fc2=weight_Variable([1024,10])
bias_fc2=bias_Variable([10])
fc2=tf.nn.softmax(tf.matmul(fc1_droput,W_fc2)+bias_fc2)

cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(fc2),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correction_prediction=tf.equal(tf.argmax(fc2,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correction_prediction,tf.float32))

sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

steps=5
for i in range(steps):
    batch=mnist.train.next_batch(batch_size)
    if i % 100 == 0:
        train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1]})
        print("step %d,train accuracy %g" %(i,train_accuracy))
        train_step.run(feed_dict={x:batch[0],y_:batch[1]})
        
print("test accuracy %g" %accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels}))


step 0,train accuracy 0.1
test accuracy 0.1014


In [34]:
# L2正则损失防止过拟合
# 获取一层神经网络边上的权重，并将这个权重的L2正则化损失加入名称为'losses的集合中'
def get_weight(shape,lambda1):
# 生成一个变量
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
# add_to_collection函数将这个新生变量的L2正则化损失加入到集合
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(var))
# 返回生成的变量
    return var


batch_size=10
x=tf.placeholder(tf.float32,[None,784])
y_=tf.placeholder(tf.float32,[None,10])
# x_image=tf.reshape(x,[-1,28,28,1])
# 定义了每一层网络中节点的个数
layer_dimension=[784,64,1024,10]
# 神经网络的层数
n_layers=len(layer_dimension)

#这个变量维护前向传播时最深层的节点，开始的时候就是输入层
cur_layer=x
# 当前层的节点的个数
in_dimension=layer_dimension[0]

# 通过一个循环来生成5层全连接层的神经网络结构
for i in range(1,n_layers):
    #layer_dimension[i]为下一层的节点个数
    out_dimension=layer_dimension[i]
#     生成当前权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    weight=get_weight([in_dimension,out_dimension],0.001)
    bias=tf.Variable(tf.constant(0.1,shape=[out_dimension]))
#     使用RELU激活函数
    cur_layer=tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
#     cur_layer=tf.nn.max_pool(cur_layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#     进入下一层之前将下一层的节点个数更新为当前层节点个数
    in_dimension=layer_dimension[i]
    total_x=mnist.train.images.shape[0]
    y=cur_layer
# 在定义神经网络前向传播的同时已经将所有的L2正则化损失加入了图上的集合
# 这里只需要计算刻画模型在训练数据上表现的损失函数
# mse_loss=tf.reduce_mean(tf.square(y_-y))
print(total_x,'total_x')
mse_loss=tf.reduce_sum(tf.pow(y_ - y,2))/total_x**2# mse_loss: 2832780.750000
# 将均方误差损失函数加入到损失集合
tf.add_to_collection('losses',mse_loss)
# get_collection 返回一个列表，这个列表是所有这个集合中的元素，在这个样例中
# 这些元素就是损失函数的不同部分,将它们加起来就可以得到最终的损失函数
loss=tf.add_n(tf.get_collection('losses'))

# 4. 训练不带正则项的损失函数mse_loss。
# 定义训练的目标函数mse_loss，训练次数及训练模型
train_op = tf.train.AdamOptimizer(0.001).minimize(mse_loss)
TRAINING_STEPS =100

with tf.Session() as sess:
    tf.global_variables_initializer().run() # 初始化所有的变量
    for i in range(TRAINING_STEPS):
        sess.run(train_op, feed_dict={x: mnist.train.images, y_: mnist.train.labels})
        if i % 100 == 0:
            print("After %d steps,\
                  mse_loss: %f" % (i,sess.run(mse_loss, feed_dict={x: mnist.train.images, y_: mnist.train.labels})))



55000 total_x
After 0 steps,                  mse_loss: 204.040558
